In [ ]:
%load_ext autoreload
%autoreload 2
from mlp2 import *
import numpy as np
from os import listdir
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
datasets = {}
path = 'project-1-part-1-data'
for file in listdir(path):
    datasets[file[:-4]] = np.loadtxt(path+'/'+file, delimiter=',', skiprows=1)

In [ ]:
datasets.keys()

In [ ]:
X_train = datasets['data.simple.train.10000'][:,:-1]
T_tr = datasets['data.simple.train.10000'][:,-1].astype(int)
X_test = datasets['data.simple.test.10000'][:,:-1]
T_tst = datasets['data.simple.test.10000'][:,-1].astype(int)

In [ ]:
px.scatter(x=X_train[:, 0], y=X_train[:, 1], color=T_tr)

In [ ]:
T_train = np.zeros((T_tr.size, T_tr.max()))
T_train[np.arange(T_tr.size),T_tr-1] = 1

In [ ]:
T_test = np.zeros((T_tst.size, T_tst.max()))
T_test[np.arange(T_tst.size),T_tst-1] = 1

In [ ]:
X_train.shape[1]

In [ ]:
T_test.shape[1]

In [ ]:
np.seterr(all='raise', under='warn', over='warn')
rng = np.random.RandomState(1234)
net = MLP(rng=rng,
          n_inputs= X_train.shape[1],
          layers=None, 
          layers_default=[3, 10, 2, SigmaLayer], 
          bias=True, 
          batch_size=32, 
          n_epochs=500, 
          eta=0.01, 
          momentum=0.9, 
          classification=True,
          learning_history=True
         )

In [ ]:
output =  net.train(X_train, 
          T_train, 
          X_test=X_test, 
          T_test=T_test, 
          verbose=False)

In [ ]:
output.keys()

In [ ]:
fig = go.Figure()
for i in range(1,5):
    fig.add_trace(
        go.Scatter(y=output[f'Linear_{i}_mean_weight'], name=f'layer {i}')
    )
fig.update_layout(xaxis_title='Epoch', yaxis_title='mean weight / initial mean weight')
fig

In [ ]:
fig = go.Figure()
for i in range(1,5):
    fig.add_trace(
        go.Scatter(y=output[f'Linear_{i}_mean_update'], name=f'layer {i}')
    )
fig.update_layout(xaxis_title='Epoch', yaxis_title='mean update')
fig

In [ ]:
go.Figure(
    [
        go.Scatter(y=output['acc_train'], name='train accuracy'),
        go.Scatter(y=output['acc_test'], name='test accuracy'),
    ]
)

In [ ]:
xx = np.linspace(-1, 1, 100)
yy = np.linspace(-1, 1, 100)
xx, yy = np.meshgrid(xx, yy)
X_sample = np.array([xx.flatten(), yy.flatten()]).T

In [ ]:
y = net.propagate(X_sample)

In [ ]:
go.Figure(
    [go.Contour(x=xx.flatten(), y=yy.flatten(), z=y[:, 0], 
                contours_showlabels=True, contours_coloring ='heatmap'),
    go.Scatter(x=X_train[:,0], y=X_train[:,1], marker_size=2,
               marker_color=T_tr, mode='markers', opacity=0.7)]
)

# Regression

In [335]:
# x = np.random.uniform(-3, 3, size=10000)
# y = np.random.uniform(-3, 3, size=10000)
x = np.random.randn(10000)
y = np.random.randn(10000)
X = np.vstack([x, y]).T
# z = np.sin(x) * np.cos(y) + np.random.randn(10000) / 100
z = np.cos(x) * np.sin(y) #+ np.random.randn(10000) / 100
z = z.reshape([len(z), 1])
X_train = X[:9000]
y_train = z[:9000]
X_test = X[9000:]
y_test = z[9000:]
px.scatter(x=x, y=y, color=z[:, 0])

In [344]:
rng = np.random.RandomState(123)
layers = [
    LinearLayer(n_inputs=2, n_units=40, rng=rng, bias=True, name='Linear_1'),
    ReLULayer(name='ReLU_1'),
    LinearLayer(n_inputs=40, n_units=40, rng=rng, bias=True, name='Linear_2'),
    ReLULayer(name='ReLU_2'),
    LinearLayer(n_inputs=40, n_units=40, rng=rng, bias=True, name='Linear_3'),
    ReLULayer(name='ReLU_3'),
    LinearLayer(n_inputs=40, n_units=1, rng=rng, bias=True, name='Linear_OUT'),
]

np.seterr(all='raise', under='warn', over='warn')
net = MLP(rng=rng,
          n_inputs= X_train.shape[1],
          layers=layers, 
          layers_default=None, 
          bias=True, 
          batch_size=32, 
          n_epochs=200, 
          eta=0.01, 
          momentum=0, 
          classification=False,
          learning_history=True
         )

In [345]:
output =  net.train(X_train, 
          y_train, 
          X_test=X_test, 
          T_test=y_test, 
          verbose=False)

epoch: initial, loss: 1.553138957438157/1.5130652123536792
epoch: 1, loss: 0.048431888675956804/0.049076973064367806
epoch: 2, loss: 0.039130736206036174/0.03926966588200261
epoch: 3, loss: 0.0330981291821898/0.03289722185372738
epoch: 4, loss: 0.0282972213929922/0.02775120717840314
epoch: 5, loss: 0.023815959889041633/0.023233005180712852
epoch: 6, loss: 0.019777515965756203/0.0193486889754234
epoch: 7, loss: 0.016575755481420484/0.016244737169973773
epoch: 8, loss: 0.01401794447896129/0.013565208004617841
epoch: 9, loss: 0.010921377210886773/0.010857594856488791
epoch: 10, loss: 0.009250722751648859/0.009254400543131862
epoch: 11, loss: 0.008037886231207964/0.007986241816426999
epoch: 12, loss: 0.007109232264352687/0.00697055581181981
epoch: 13, loss: 0.006355122189112011/0.006178231171677396
epoch: 14, loss: 0.0057097669541886645/0.0055000401200043176
epoch: 15, loss: 0.005112841487670764/0.00491447765065969
epoch: 16, loss: 0.004631890323279616/0.004442452721938334
epoch: 17, loss:

epoch: 136, loss: 0.0007691853790112554/0.00131732466190612
epoch: 137, loss: 0.0007641467403111098/0.001307476692257629
epoch: 138, loss: 0.0007603754640412921/0.0013047929054084528
epoch: 139, loss: 0.0007570909943941806/0.0013077755861296359
epoch: 140, loss: 0.0007534263618706866/0.0013015000459447076
epoch: 141, loss: 0.0007504061828038012/0.001296917547492059
epoch: 142, loss: 0.0007473075597114963/0.0012922058287778522
epoch: 143, loss: 0.0007449576276721152/0.0012881888270030752
epoch: 144, loss: 0.0007424581558139162/0.0012879148380448007
epoch: 145, loss: 0.0007381592702579004/0.0012844176124126466
epoch: 146, loss: 0.0007350326613946003/0.0012808834397842524
epoch: 147, loss: 0.0007325310396403053/0.0012788743712160758
epoch: 148, loss: 0.0007295036815114988/0.00127569617169031
epoch: 149, loss: 0.0007266354545445874/0.0012730513257956772
epoch: 150, loss: 0.0007231672476349339/0.001269458218296002
epoch: 151, loss: 0.0007205053734250533/0.0012671307596578744
epoch: 152, los

In [346]:
fig = go.Figure()
for i in range(1,5):
    fig.add_trace(
        go.Scatter(y=output[f'Linear_{i}_mean_weight'], name=f'layer {i}')
    )
fig.update_layout(xaxis_title='Epoch', yaxis_title='mean weight / initial mean weight')
fig

In [353]:
xx = np.linspace(-3, 3, 100)
yy = np.linspace(-3, 3, 100)
xxx, yyy = np.meshgrid(xx, yy)
X_sample = np.array([xxx.flatten(), yyy.flatten()]).T

In [354]:
y = net.propagate(X_sample)

In [355]:
go.Figure(data=[
    go.Surface(x=xxx, y=yyy, z=y.reshape(100, 100)),
])